利用[Ames Housing dataset](https://ww2.amstat.org/publications/jse/v19n3/decock.pdf)預測房價


* Data type tranformation
* Imputation
* Create/Discard Features
* Normality
* Outlier Detection
* Model Selection

In [1]:
# -*- coding: utf-8 -*-

# pandas
import pandas as pd

# numpy, matplotlb, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import scipy.stats as st

In [2]:
# Import Dataset
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

df = train_df.append(test_df, ignore_index=True) # train = df[:1460], test = df[1460:]
df = df.drop('Id', axis=1)

## Baseline Model
A baseline model against to our feature engineering results.

### Preprocess

In [3]:
def preScoreTest(df):
    train = df[:1460]
    test = df[1460:]
    
    train = pd.get_dummies(train)
    train = train.fillna(train.median())
    
    test = pd.get_dummies(test)
    test = test.fillna(test.median())
    
    common_cols = list(set(train.columns.tolist()) & set(test.columns.tolist()))
    test = test[common_cols]
    test = test.drop('SalePrice', axis=1)
    train = train[common_cols]
    
    return train, test

In [4]:
train, test = preScoreTest(df)

### Modelling

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

rfg = RandomForestRegressor(n_estimators=300, random_state=0, n_jobs=-1)
accuracies = cross_val_score(estimator = rfg, X = train.drop('SalePrice', axis=1), y = train['SalePrice'], cv = 10, n_jobs=-1)
print('CV score is: ', accuracies.mean())

CV score is:  0.863350385716


In [6]:
rfg.fit(train.drop('SalePrice', axis=1), y=train['SalePrice'])
y_pred = rfg.predict(test)
submission = pd.DataFrame({
    'Id':test_df['Id'],
    'SalePrice':y_pred
})
submission.to_csv('sub_base_test', index=False)

### Data Type Transformation
* Numeric to Categoric  
* Rank Fields to Numeric  

In [7]:
df['MSSubClass'] = df['MSSubClass'].astype('object')

In [8]:
df.BsmtCond =  df.BsmtCond.map({'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.BsmtExposure = df.BsmtExposure.map({'NA':0, 'No':1, 'Mn':2, 'Av':3, 'Gd':4})
df.BsmtFinType1 = df.BsmtFinType1.map({'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6})
df.BsmtFinType2 = df.BsmtFinType2.map({'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6})
df.BsmtQual = df.BsmtQual.map({'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})

df.ExterCond = df.ExterCond.map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.ExterQual = df.ExterQual.map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})

df.FireplaceQu = df.FireplaceQu.map({'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})

df.GarageCond = df.GarageCond.map({'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.GarageQual = df.GarageQual.map({'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})

df.HeatingQC = df.HeatingQC.map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.KitchenQual = df.KitchenQual.map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.PoolQC = df.PoolQC.map({'NA':0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4})

In [9]:
train, test = preScoreTest(df)

rfg = RandomForestRegressor(n_estimators=300, random_state=0, n_jobs=-1)
accuracies = cross_val_score(estimator = rfg, X = train.drop('SalePrice', axis=1), y = train['SalePrice'], cv = 10, n_jobs=-1)
print('CV score is:', accuracies.mean())

CV score is: 0.865779126938


### Imputation

In [10]:
# Missing data in train set
def missing_data(df):
    missing = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum() / df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([missing, percent], axis=1, keys=['Missing', 'Percent'])
    return missing_data
missing = missing_data(df)
print(missing.head(35))

              Missing   Percent
PoolQC           2909  0.996574
MiscFeature      2814  0.964029
Alley            2721  0.932169
Fence            2348  0.804385
SalePrice        1459  0.499829
FireplaceQu      1420  0.486468
LotFrontage       486  0.166495
GarageQual        159  0.054471
GarageYrBlt       159  0.054471
GarageFinish      159  0.054471
GarageCond        159  0.054471
GarageType        157  0.053786
BsmtCond           82  0.028092
BsmtExposure       82  0.028092
BsmtQual           81  0.027749
BsmtFinType2       80  0.027407
BsmtFinType1       79  0.027064
MasVnrType         24  0.008222
MasVnrArea         23  0.007879
MSZoning            4  0.001370
BsmtHalfBath        2  0.000685
Utilities           2  0.000685
Functional          2  0.000685
BsmtFullBath        2  0.000685
Electrical          1  0.000343
Exterior2nd         1  0.000343
KitchenQual         1  0.000343
GarageCars          1  0.000343
Exterior1st         1  0.000343
GarageArea          1  0.000343
TotalBsm

* PoolQC, MiscFeature, Alley, Fence, FireplaceQu, GarageX, BsmtX都是由於沒有(no Pool ,Miss Feature, Alley ...)，而非真的遺失因此填上"No"值。
* 利用其他欄位預測LotFrontage的遺失值
* 其餘遺失值都很低(<0.3%)，因此帶入最常見的數值

In [11]:
df_m = df.copy()

In [12]:
df_m['PoolQC'].fillna('No', inplace=True)
df_m['Alley'].fillna('No', inplace=True)
df_m['MiscFeature'].fillna('No', inplace=True)
df_m['Fence'].fillna('No', inplace=True)
df_m['FireplaceQu'].fillna('No', inplace=True)

df_m['Fence'].fillna('No', inplace=True)

df_m['GarageFinish'].fillna('No', inplace=True)
df_m['GarageType'].fillna('No', inplace=True)
df_m['GarageYrBlt'].fillna(0, inplace=True) 
df_m['GarageCond'].fillna(0, inplace=True)
df_m['GarageQual'].fillna(0, inplace=True)
df_m['GarageCars'].fillna(0, inplace=True)
df_m['GarageArea'].fillna(0, inplace=True)

df_m['BsmtFinType2'].fillna(0, inplace=True)
df_m['BsmtExposure'].fillna(0, inplace=True)
df_m['BsmtFinType1'].fillna(0, inplace=True)
df_m['BsmtQual'].fillna(0, inplace=True)
df_m['BsmtCond'].fillna(0, inplace=True)
df_m['BsmtHalfBath'].fillna(0, inplace=True)
df_m['BsmtFullBath'].fillna(0, inplace=True)
df_m['BsmtFinSF1'].fillna(0, inplace=True)
df_m['BsmtFinSF2'].fillna(0, inplace=True)
df_m['TotalBsmtSF'].fillna(0, inplace=True)
df_m['BsmtUnfSF'].fillna(0, inplace=True)

df_m['MasVnrArea'].fillna(0, inplace=True)
df_m['MasVnrType'].fillna('No', inplace=True)

df_m['Electrical'].fillna('SBrkr', inplace=True)
df_m['MSZoning'].fillna('RL', inplace=True)
df_m['Functional'].fillna('Typ', inplace=True)
df_m['SaleType'].fillna('WD', inplace=True)
df_m['KitchenQual'].fillna(3, inplace=True)
df_m['Exterior1st'].fillna('VinylSd', inplace=True)
df_m['Exterior2nd'].fillna('VinylSd', inplace=True)

df_m = df_m.drop('Utilities', axis=1)

In [13]:
# Using xgboost to impute missing LotFrontage
from xgboost import XGBRegressor

null_idx = df_m.loc[df_m['LotFrontage'].isnull()].index

df_lot = df_m.copy()
df_lot = pd.get_dummies(df_lot)
train_lot = df_lot.drop(df_lot.index[[null_idx]])
test_lot = df_lot.loc[null_idx]

test_lot.shape, train_lot.shape

xgb = XGBRegressor(learning_rate=0.05,n_estimators=500,max_depth=3,colsample_bytree=0.4)
xgb.fit(train_lot.drop(['LotFrontage', 'SalePrice'], axis=1), train_lot['LotFrontage'])
pred_lot = xgb.predict(test_lot.drop(['LotFrontage', 'SalePrice'], axis=1))
df_m.loc[null_idx, 'LotFrontage'] = pred_lot

/home/lafes/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
missing = missing_data(df_m)
print(missing.head(5))

            Missing   Percent
SalePrice      1459  0.499829
YrSold            0  0.000000
Foundation        0  0.000000
ExterCond         0  0.000000
ExterQual         0  0.000000


SalePrice missing at test.csv

In [15]:
train, test = preScoreTest(df_m)

rfg = RandomForestRegressor(n_estimators=300, random_state=0, n_jobs=-1)
accuracies = cross_val_score(estimator = rfg, X = train.drop('SalePrice', axis=1), y = train['SalePrice'], cv = 10, n_jobs=-1)
print('CV score is:', accuracies.mean())

CV score is: 0.865058656616


## XGBoost!

In [16]:
from xgboost import XGBRegressor
xgb = XGBRegressor(learning_rate=0.05,n_estimators=500,max_depth=3,colsample_bytree=0.4)
accuracies = cross_val_score(estimator = xgb, X = train.drop('SalePrice', axis=1), y = train['SalePrice'], cv = 5, n_jobs=-1)
print('CV score is:', accuracies.mean())

CV score is: 0.899458294302


## Create / Discard Features

In [17]:
df_f = df_m.copy()

In [18]:
# Too much duplicate value in single column
dup = [n for n in df_f.columns if df_f[n].value_counts().max()/len(df_f) > 0.97]
print(dup)

['3SsnPorch', 'Condition2', 'Heating', 'LowQualFinSF', 'PoolArea', 'PoolQC', 'RoofMatl', 'Street']


In [19]:
df_f = df_f.drop(dup, axis=1)

In [20]:
# Total sq
df_f['TotalSF'] = df_f['TotalBsmtSF'] + df_f['1stFlrSF'] + df_f['2ndFlrSF']

In [21]:
# Bath
df_f['Bath'] = df_f['FullBath'] + 0.5 * df_f['HalfBath'] + df_f['BsmtFullBath'] + 0.5 * df_f['BsmtHalfBath']
df_f = df_f.drop(['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath'], axis=1)

In [22]:
train, test = preScoreTest(df_f)

xgb = XGBRegressor(learning_rate=0.05,n_estimators=500,max_depth=3,colsample_bytree=0.4)
accuracies = cross_val_score(estimator = xgb, X = train.drop('SalePrice', axis=1), y = train['SalePrice'], cv = 10, n_jobs=-1)
print('CV score is:', accuracies.mean())

CV score is: 0.907265973702


## Normality

In [23]:
df_normal = df_f.copy()

In [24]:
# skewness of features
numeric_cols = [f for f in df_normal if df_normal.dtypes[f] != 'object']
df_normal[numeric_cols].skew()

1stFlrSF          1.470360
2ndFlrSF          0.862118
BedroomAbvGr      0.326492
BsmtCond         -3.605964
BsmtExposure      1.117303
BsmtFinSF1        1.425963
BsmtFinSF2        4.148275
BsmtFinType1     -0.149621
BsmtFinType2      3.153959
BsmtQual         -1.269195
BsmtUnfSF         0.919812
EnclosedPorch     4.005950
ExterCond         1.316590
ExterQual         0.786786
Fireplaces        0.733872
GarageArea        0.239380
GarageCars       -0.219694
GarageCond       -3.384860
GarageQual       -3.265354
GarageYrBlt      -3.908213
GrLivArea         1.270010
HeatingQC        -0.550192
KitchenAbvGr      4.304467
KitchenQual       0.438761
LotArea          12.829025
LotFrontage       1.315325
MasVnrArea        2.614936
MiscVal          21.958480
MoSold            0.195985
OpenPorchSF       2.536417
OverallCond       0.570605
OverallQual       0.197212
SalePrice         1.882876
ScreenPorch       3.948723
TotRmsAbvGrd      0.758757
TotalBsmtSF       1.157489
WoodDeckSF        1.843380
Y

In [25]:
skew =df_normal[numeric_cols].columns[df_normal[numeric_cols].skew() > 0.75]
skew

Index(['1stFlrSF', '2ndFlrSF', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtFinType2', 'BsmtUnfSF', 'EnclosedPorch', 'ExterCond', 'ExterQual',
       'GrLivArea', 'KitchenAbvGr', 'LotArea', 'LotFrontage', 'MasVnrArea',
       'MiscVal', 'OpenPorchSF', 'SalePrice', 'ScreenPorch', 'TotRmsAbvGrd',
       'TotalBsmtSF', 'WoodDeckSF', 'TotalSF'],
      dtype='object')

In [26]:
for i in skew.values:
    df_normal[i] = np.log1p(df_normal[i])

In [27]:
train, test = preScoreTest(df_normal)

xgb = XGBRegressor(learning_rate=0.05,n_estimators=500,max_depth=3,colsample_bytree=0.4)
accuracies = cross_val_score(estimator = xgb, X = train.drop('SalePrice', axis=1), y = train['SalePrice'], cv = 10, n_jobs=-1)
print('CV score is:', accuracies.mean())

CV score is: 0.914369489849


In [28]:
rfg.fit(train.drop('SalePrice', axis=1), y=train['SalePrice'])
y_pred = np.expm1(rfg.predict(test))
submission = pd.DataFrame({
    'Id':test_df['Id'],
    'SalePrice':y_pred
})
submission.to_csv('sub_normal', index=False)

### Outlier Detection

In [29]:
df_out = df_normal.copy()

In [30]:
import statsmodels.api as sm
train, test = preScoreTest(df_out)
regressor = sm.OLS(train['SalePrice'], train.drop('SalePrice', axis=1)).fit()
result = regressor.outlier_test()

/home/lafes/anaconda3/lib/python3.6/site-packages/statsmodels/stats/outliers_influence.py:309: RuntimeWarning: invalid value encountered in sqrt
  return  self.results.resid / sigma / np.sqrt(1 - hii)
/home/lafes/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/lafes/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/lafes/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
/home/lafes/anaconda3/lib/python3.6/site-packages/statsmodels/stats/multitest.py:147: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
/home/lafes/anaconda3/lib/python3.6/site-packages/statsmodels/stats/multitest.py:251

In [31]:
outlier = result['bonf(p)']
outlier = outlier[outlier<1e-3].index
print(outlier)

Int64Index([462, 523, 632, 968, 1298, 1324], dtype='int64')


In [32]:
train = train.drop(outlier, axis=0)

In [33]:
xgb = XGBRegressor(learning_rate=0.05,n_estimators=500,max_depth=3,colsample_bytree=0.4)
accuracies = cross_val_score(estimator = xgb, X = train.drop('SalePrice', axis=1), y = train['SalePrice'], cv = 10, n_jobs=-1)
print('CV score is:', accuracies.mean())

CV score is: 0.926450647868


## Model Selection

In [34]:
X_train = train.drop('SalePrice', axis=1)
y_train = train['SalePrice']
X_test = test

### Baseline Model

In [35]:
from sklearn.linear_model import ElasticNet, Lasso, OrthogonalMatchingPursuit
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

In [36]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [38]:
lasso = Lasso(alpha =0.001, random_state=1)
ENet = ElasticNet(alpha=0.001, l1_ratio=.9, random_state=3)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
xgb = XGBRegressor(colsample_bytree=0.2, learning_rate=0.05, max_depth=3, n_estimators=1200)
orth = OrthogonalMatchingPursuit()

In [39]:
def scoreTest(models):
    score = {}
    predict = {}
    for m in models:
        name = m.__class__.__name__
        mean = rmsle_cv(m).mean()
        score[name] = mean
        
        m.fit(X_train, y_train)
        y_pred = m.predict(test)
        
        predict[name] = y_pred
    score = pd.Series(score)
    predict = pd.DataFrame(predict)
    return score, predict

In [41]:
result, predict = scoreTest(models = [lasso ,ENet, GBoost, xgb, orth])

/home/lafes/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/lafes/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/lafes/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/lafes/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not conver

In [42]:
predict.corr()

,ElasticNet,GradientBoostingRegressor,Lasso,OrthogonalMatchingPursuit,XGBRegressor
ElasticNet,1.000000,0.984429,0.999970,0.989629,0.989370
GradientBoostingRegressor,0.984429,1.000000,0.984336,0.972248,0.991722
Lasso,0.999970,0.984336,1.000000,0.989503,0.989106
OrthogonalMatchingPursuit,0.989629,0.972248,0.989503,1.000000,0.980079
XGBRegressor,0.989370,0.991722,0.989106,0.980079,1.000000


In [43]:
result.sort_values(ascending=True)

XGBRegressor                 0.104689
ElasticNet                   0.106845
Lasso                        0.107447
GradientBoostingRegressor    0.108687
OrthogonalMatchingPursuit    0.119163
dtype: float64

In [45]:
y_pred = np.expm1(0.25*predict['ElasticNet'] + 0.2 * predict['GradientBoostingRegressor'] + 0.25*predict['Lasso'] + 0.3*predict['XGBRegressor'])

In [46]:
submission = pd.DataFrame({
    'Id':test_df['Id'],
    'SalePrice':y_pred
})
submission.to_csv('avg_ensemble', index=False)